In [ ]:
let ( >> ) f g x = g (f x)

let ( << ) f g x = f (g x)

let id x = x

let const x _ = x

In [ ]:
module type Monad = sig
  type 'a t

  val return : 'a -> 'a t

  val bind : 'a t -> ('a -> 'b t) -> 'b t

  (* infix operator for bind *)
  val ( >>= ) : 'a t -> ('a -> 'b t) -> 'b t

  (* infix operator for bind *)
  val ( let* ) : 'a t -> ('a -> 'b t) -> 'b t
end

In [202]:
module ListMonad : Monad with type 'a t = 'a list = struct
  type 'a t = 'a list

  let return x = [ x ]

  let bind l f = List.concat_map f l

  let ( >>= ) = bind

  let ( let* ) = bind
end

module ListMonad :
  sig
    type 'a t = 'a list
    val return : 'a -> 'a t
    val bind : 'a t -> ('a -> 'b t) -> 'b t
    val ( >>= ) : 'a t -> ('a -> 'b t) -> 'b t
    val ( let* ) : 'a t -> ('a -> 'b t) -> 'b t
  end


In [203]:
module type Functor = sig
  type 'a t

  val fmap : ('a -> 'b) -> 'a t -> 'b t

  (* infix operator for fmap *)
  val ( <!> ) : ('a -> 'b) -> 'a t -> 'b t
  
  (* infix operator for fmap *)
  val ( let+ ) : 'a t -> ('a -> 'b) -> 'b t
end

module type Functor =
  sig
    type 'a t
    val fmap : ('a -> 'b) -> 'a t -> 'b t
    val ( <!> ) : ('a -> 'b) -> 'a t -> 'b t
    val ( let+ ) : 'a t -> ('a -> 'b) -> 'b t
  end


In [204]:
module ListFunctor : Functor with type 'a t = 'a list = struct
  type 'a t = 'a list

  let fmap f l =
    let rec aux acc = function
      | [] -> List.rev acc
      | x :: xs -> aux (f x :: acc) xs
    in
    aux [] l

  let ( <!> ) = fmap

  let ( let+ ) x f = fmap f x
end

module ListFunctor :
  sig
    type 'a t = 'a list
    val fmap : ('a -> 'b) -> 'a t -> 'b t
    val ( <!> ) : ('a -> 'b) -> 'a t -> 'b t
    val ( let+ ) : 'a t -> ('a -> 'b) -> 'b t
  end


In [205]:
module type Applicative = sig
  type 'a t

  val apply : ('a -> 'b) t -> 'a t -> 'b t

  val product : 'a t -> 'b t -> ('a * 'b) t

  (* infix operator for apply *)
  val ( <*> ) : ('a -> 'b) t -> 'a t -> 'b t

  (* infix operator for product *)
  val ( and+ ) : 'a t -> 'b t -> ('a * 'b) t

  val ( <* ) : 'a t -> 'b t -> 'a t

  val ( *> ) : 'a t -> 'b t -> 'b t

  val lift2 : ('a -> 'b -> 'c) -> 'a t -> 'b t -> 'c t

  val lift3 : ('a -> 'b -> 'c -> 'd) -> 'a t -> 'b t -> 'c t -> 'd t
end

module type Applicative =
  sig
    type 'a t
    val apply : ('a -> 'b) t -> 'a t -> 'b t
    val product : 'a t -> 'b t -> ('a * 'b) t
    val ( <*> ) : ('a -> 'b) t -> 'a t -> 'b t
    val ( and+ ) : 'a t -> 'b t -> ('a * 'b) t
    val ( <* ) : 'a t -> 'b t -> 'a t
    val ( *> ) : 'a t -> 'b t -> 'b t
    val lift2 : ('a -> 'b -> 'c) -> 'a t -> 'b t -> 'c t
    val lift3 : ('a -> 'b -> 'c -> 'd) -> 'a t -> 'b t -> 'c t -> 'd t
  end


In [206]:
module ListApplicative : Applicative with type 'a t = 'a list =
struct
  include ListFunctor

  type 'a t = 'a list

  let pure a = [ a ]

  let apply fs ls = List.concat_map (fun f -> List.map (fun x -> f x) ls) fs

  let product fs ls =
    List.concat_map (fun f -> List.map (fun x -> (f, x)) ls) fs

  let ( <*> ) = apply

  let ( and+ ) = product

  let ( <* ) l1 l2 = const <!> l1 <*> l2

  let ( *> ) l1 l2 = (fun _ y -> y) <!> l1 <*> l2

  let lift2 f l1 l2 = f <!> l1 <*> l2

  let lift3 f l1 l2 l3 = f <!> l1 <*> l2 <*> l3
end

module ListApplicative :
  sig
    type 'a t = 'a list
    val apply : ('a -> 'b) t -> 'a t -> 'b t
    val product : 'a t -> 'b t -> ('a * 'b) t
    val ( <*> ) : ('a -> 'b) t -> 'a t -> 'b t
    val ( and+ ) : 'a t -> 'b t -> ('a * 'b) t
    val ( <* ) : 'a t -> 'b t -> 'a t
    val ( *> ) : 'a t -> 'b t -> 'b t
    val lift2 : ('a -> 'b -> 'c) -> 'a t -> 'b t -> 'c t
    val lift3 : ('a -> 'b -> 'c -> 'd) -> 'a t -> 'b t -> 'c t -> 'd t
  end


In [207]:
module List_ex = struct
    include Stdlib.List
    include ListFunctor
    include ListApplicative
    include ListMonad
end;;

module List_ex :
  sig
    val length : 'a list -> int
    val compare_lengths : 'a list -> 'b list -> int
    val compare_length_with : 'a list -> int -> int
    val cons : 'a -> 'a list -> 'a list
    val hd : 'a list -> 'a
    val tl : 'a list -> 'a list
    val nth : 'a list -> int -> 'a
    val nth_opt : 'a list -> int -> 'a option
    val rev : 'a list -> 'a list
    val init : int -> (int -> 'a) -> 'a list
    val append : 'a list -> 'a list -> 'a list
    val rev_append : 'a list -> 'a list -> 'a list
    val concat : 'a list list -> 'a list
    val flatten : 'a list list -> 'a list
    val equal : ('a -> 'a -> bool) -> 'a list -> 'a list -> bool
    val compare : ('a -> 'a -> int) -> 'a list -> 'a list -> int
    val iter : ('a -> unit) -> 'a list -> unit
    val iteri : (int -> 'a -> unit) -> 'a list -> unit
    val map : ('a -> 'b) -> 'a list -> 'b list
    val mapi : (int -> 'a -> 'b) -> 'a list -> 'b list
    val rev_map : ('a -> 'b) -> 'a list -> 'b list
    val filter_ma

In [214]:
open List_ex;;
let nums = [1;2;3;4;5];;
let letters = ['a'; 'b'; 'c'];;
let r1 = 
    nums >>= fun n ->
    letters >>= fun l ->
    return (n, l);;
let r2 =
    let* n = nums in
    let* l = letters in
    return (n,l);;
let r3 =
    (fun x y -> (x,y)) <!> nums <*> letters;;
let r4 =
    let+ x = nums 
    and+ y = letters in
    (x,y);;
r1 = r2 && r1 = r3 && r1 = r4;;
[1;2;3] <* [3;4;5] ;;
[1;2;3] *> [3;4;5] ;;

let repeat n pattern =
    (List.init n (fun x -> x)) *> pattern;;
let replicate n x =
    repeat n [x];;
    
repeat 10 ["a";"b"];;
replicate 10 'R';;

let add10 x = x + 10;;
let add20 x = x + 20;;
[add10; add20] <*> nums;;

lift2 (fun x y -> x + y) [10;20] nums ;;

val nums : int list = [1; 2; 3; 4; 5]


val letters : char list = ['a'; 'b'; 'c']


val r1 : (int * char) List_ex.t =
  [(1, 'a'); (1, 'b'); (1, 'c'); (2, 'a'); (2, 'b'); (2, 'c'); (3, 'a');
   (3, 'b'); (3, 'c'); (4, 'a'); (4, 'b'); (4, 'c'); (5, 'a'); (5, 'b');
   (5, 'c')]


val r2 : (int * char) List_ex.t =
  [(1, 'a'); (1, 'b'); (1, 'c'); (2, 'a'); (2, 'b'); (2, 'c'); (3, 'a');
   (3, 'b'); (3, 'c'); (4, 'a'); (4, 'b'); (4, 'c'); (5, 'a'); (5, 'b');
   (5, 'c')]


val r3 : (int * char) list =
  [(1, 'a'); (1, 'b'); (1, 'c'); (2, 'a'); (2, 'b'); (2, 'c'); (3, 'a');
   (3, 'b'); (3, 'c'); (4, 'a'); (4, 'b'); (4, 'c'); (5, 'a'); (5, 'b');
   (5, 'c')]


...

- : char list = ['R'; 'R'; 'R'; 'R'; 'R'; 'R'; 'R'; 'R'; 'R'; 'R']


val add10 : int -> int = <fun>


val add20 : int -> int = <fun>


- : int list = [11; 12; 13; 14; 15; 21; 22; 23; 24; 25]


- : int list = [11; 12; 13; 14; 15; 21; 22; 23; 24; 25]


In [ ]:
let n1 = [10;100];;
let n2 = [1;2;3];;
[(+)] <*> n1 <*> n2;;
(+) <!> n1 <*> n2;;
let+ x = n1 and+ y = n2 in x + y;;

In [1]:
let add a b = a + b;;
add 1 2;;

val add : int -> int -> int = <fun>


- : int = 3
